In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import node2vec
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm
from collections import defaultdict
import itertools

In [19]:
df = pq.read_table('Downloads/data_made_restaurants.parquet').to_pandas()

In [21]:
df[df.city_id == 2].shape

(359109, 25)

In [3]:
df.date.min(), df.date.max()

('2020-08-01', '2020-10-31')

In [4]:
df[df.date < '2020-09-01']

,Unnamed: 0,customer_id,order_id,user_latitude,user_longitude,date,city_id,chain_id,vendor_id,target,...,vendor_latitude,vendor_longitude,online_payment,accepting_cash,min_delivery_value,takeaway_support,citymobil_support,default_product_group_id,product_group_ids,cuisine_ids
0,0,15955880,207845807,55.7815,37.5307,2020-08-01,1,140718,343852,1,...,55.778137,37.602400,1,0,1000,1,0,30.0,[30],None
1,1,62512097,207855295,55.6472,37.4682,2020-08-01,1,140718,343852,1,...,55.778137,37.602400,1,0,1000,1,0,30.0,[30],None
2,2,64977556,207871966,55.8649,37.5014,2020-08-01,1,140718,343852,1,...,55.778137,37.602400,1,0,1000,1,0,30.0,[30],None
3,3,81281415,207960541,55.8711,37.5105,2020-08-01,1,140718,343852,1,...,55.778137,37.602400,1,0,1000,1,0,30.0,[30],None
4,4,72045218,208028305,55.8166,37.5899,2020-08-01,1,140718,343852,1,...,55.778137,37.602400,1,0,1000,1,0,30.0,[30],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2688520,2688520,11758480,219364390,59.9496,30.3787,2020-08-31,2,184570,436906,1,...,59.943453,30.388829,1,1,500,0,0,6.0,"[6, 26]",None
2688530,2688530,83960069,219368239,55.6858,37.3070,2020-08-31,1,169006,381790,1,...,55.686147,37.303467,1,1,300,1,0,20.0,"[6, 20]",None
2688532,2688532,3301424,219368302,55.7627,37.5659,2020-08-31,1,182162,426022,1,...,55.752323,37.566764,1,1,300,0,0,20.0,[20],None
2688535,2688535,57010858,219382356,55.8262,37.9569,2020-08-31,1,48662,441406,1,...,55.805391,37.981857,1,0,600,0,0,37.0,"[1, 5, 21, 37]","[1, 4]"


In [5]:
df = df[(df.city_id == 1)]

In [55]:
df['size'] = df.groupby('customer_id')['chain_id'].transform('nunique')

In [7]:
chains = df[(df['size'] > 1) & (df['customer_id'] != -10)].groupby('customer_id')['chain_id'].apply(set)

In [8]:
d = defaultdict(int)

for row in tqdm(chains.values):
    for u, v in itertools.combinations(row, 2):
        d[(u, v)] += 1
        d[(v, u)] += 1

In [9]:
G = nx.Graph()
G.add_weighted_edges_from([k + (float(v),) for k, v in d.items()])

In [10]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 4454
Number of edges: 111842
Average degree:  50.2209


In [11]:
print(nx.number_connected_components(G))

2


In [39]:
df[df.customer_id == np.random.choice(df[df.city_id == 1].customer_id)][['date', 'chain_id', 'product_group_ids']]

,date,chain_id,product_group_ids
405213,2020-09-08,137418,None
744656,2020-10-29,117814,"[25, 33]"
1361014,2020-08-08,128194,"[24, 33, 43]"
1383609,2020-09-19,142286,"[23, 24, 34, 42]"
1887078,2020-08-30,133522,None
2417250,2020-10-09,135994,"[5, 25, 31, 48]"


In [12]:
n2v = node2vec.Node2Vec(G)

Generating walks (CPU: 1): 100%|██████████| 10/10 [11:31<00:00, 69.10s/it]


In [13]:
model = n2v.fit()

In [14]:
chain_names = df[['chain_id', 'chain_name']].groupby('chain_id')['chain_name'].unique()
chain_names = dict(chain_names)
chain_names = dict([(k, v[0]) for k, v in chain_names.items()])

In [43]:
df.chain_id.values

array([140718, 140718, 140718, ..., 198014, 200078, 166154])

In [96]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from MulticoreTSNE import MulticoreTSNE as TSNE
from bokeh.transform import factor_cmap
from bokeh.palettes import PuBu, Bokeh, Cividis256, YlOrBr
from bokeh.models import LinearColorMapper
 
tsne = TSNE()
vocab = list(map(int, model.wv.vocab))
vectors = np.vstack(list(map(model.wv.get_vector, map(str, vocab))))
Y = tsne.fit_transform(vectors)
# centrality = nx.centrality.approximate_current_flow_betweenness_centrality(G, weight='weight')
# centrality = np.array(list(map(chain_names.get, vocab)))
 
output_notebook()


fill_color=list(['yellow' for _ in range(len(vocab))])

x = df[df.customer_id == np.random.choice(df[(df.city_id == 1) & (df['size'] > 4)].customer_id)][['date', 'chain_id', 'product_group_ids', 'chain_name']]

print(x)

index = []

for i in x.chain_id.values[:-3]:
    fill_color[vocab.index(i)] = 'blue'   
    index.append(vocab.index(i))
    
for i in x.chain_id.values[-3:]:
    fill_color[vocab.index(i)] = 'red'
    index.append(vocab.index(i))
    
index = list(set(index))
                
 
source = ColumnDataSource(
        data=dict(
            x=Y[:,0],
            y=Y[:,1],
            desc=list(np.array(list(map(chain_names.get, vocab)))),
            fill_color=list(np.array(fill_color))
        )
    )

# mapper = LinearColorMapper(
#     palette='Magma256',
#     low=-centrality.max(),
#     high=-centrality.min()
# )

hover = HoverTool(
        tooltips=[
            ("desc", "@desc"),
        ]
    )
 
p = figure(plot_width=900, plot_height=700,
           tools=[hover]+"crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,".split(','),
           title="Mouse over the dots")
 
p.circle('x', 'y', size=10, source=source, fill_color='fill_color')
show(p)

Loading BokehJS ...

               date  chain_id   product_group_ids         chain_name
557175   2020-08-07    137418                None  Самокат. Продукты
557318   2020-09-03    137418                None  Самокат. Продукты
557334   2020-09-06    137418                None  Самокат. Продукты
557575   2020-10-13    137418                None  Самокат. Продукты
557637   2020-10-21    137418                None  Самокат. Продукты
557655   2020-10-24    137418                None  Самокат. Продукты
557670   2020-10-26    137418                None  Самокат. Продукты
1166143  2020-10-25    129794    [29, 34, 43, 51]    Крошка Картошка
1254527  2020-09-28    114878             [6, 26]         Макдоналдс
1377667  2020-10-05    121994                 [6]               FARШ
1521667  2020-08-27       786     [2, 18, 36, 40]          Маки Маки
1742185  2020-08-16    127102             [1, 26]     Domino’s Pizza
2493764  2020-08-13        50  [1, 2, 40, 47, 51]              Нияма
2692225  2020-10-20    127886     